## Yelp API

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re
import time
import random as ran

In [100]:
clientId = 'YOUR_CLIENT_ID'
apiKey = 'YOUR_API_KEY'

headers = {'Authorization': 'Bearer %s' % apiKey}

In [101]:
# request businesses
offset = 0
url='https://api.yelp.com/v3/businesses/search'
params = {'term':'restaurant','location':'Montreal, QC','limit':10, 'offset':offset}
r = requests.get(url, headers=headers, params=params)
j = json.loads(r.text)
print(r.status_code)

200


In [1]:
#j['businesses'][0]

In [103]:
url='https://api.yelp.com/v3/businesses/search'
offset = 0
df = pd.DataFrame()

# catch errors for missing data
def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        return np.nan

# unpack a list of dict
def unpack(e):
    try:
        return [ _['alias'] for _ in e ]
    except Exception as e:
        return np.nan

while offset < 1000:
    params = params = {'term':'restaurant','location':'Montreal, QC','limit':50, 'offset':offset} 
    r = requests.get(url, headers=headers, params=params)
    j = json.loads(r.text)
    b = j['businesses']

    biz = {}
    biz['id'] = [ catch(lambda : e['id']) for e in b ]
    biz['name'] = [ catch(lambda : e['name']) for e in b ]
    biz['url'] = [ catch(lambda : e['url'].split('?')[0]) for e in b ]
    biz['review_count'] = [ catch(lambda : e['review_count']) for e in b ]
    biz['categories'] = [ unpack(e['categories']) for e in b ]
    biz['rating'] = [ catch(lambda : e['rating']) for e in b ]
    biz['latitude'] = [ catch(lambda : e['coordinates']['latitude']) for e in b ]
    biz['longitude'] = [ catch(lambda : e['coordinates']['longitude']) for e in b ]
    biz['price'] = [ catch(lambda : e['price']) for e in b ]
    biz['location'] = [ catch(lambda : e['location']['display_address']) for e in b ]
    biz['phone'] = [ catch(lambda : e['phone']) for e in b ]
    offset += 50
    #print(offset)
    df = pd.concat([df, pd.DataFrame.from_dict(biz)])


In [3]:
#df.to_csv('data/businesses_mtl.csv', index=False)
df = pd.read_csv('data/businesses_mtl.csv')
df.sort_values(by='review_count', ascending=False)
#df

,id,name,url,review_count,categories,rating,latitude,longitude,price,location,phone
0,0W4lkclzZThpx3V65bVgig,Schwartz's,https://www.yelp.com/biz/schwartzs-montr%C3%A9...,2697,"['delis', 'sandwiches']",4.0,45.516353,-73.577642,$$,"['3895 Boulevard Saint-Laurent', 'Montreal, QC...",1.514842e+10
9,5T6kFKFycym_GkhgOiysIw,La Banquise,https://www.yelp.com/biz/la-banquise-montr%C3%...,1957,['poutineries'],4.0,45.525354,-73.574769,$$,"['994 Rue Rachel E', 'Montreal, QC H2J 2J3', '...",1.514525e+10
10,46Ld9Qc9nAx_A0jwclNZiw,Olive & Gourmando,https://www.yelp.com/biz/olive-and-gourmando-m...,1401,"['sandwiches', 'breakfast_brunch']",4.5,45.501197,-73.556475,$$,"['351 Rue Saint-Paul O', 'Montreal, QC H2Y 2A7...",1.514350e+10
11,cKdox2gt3L1Dbb7MpOPdWg,Au Pied de Cochon,https://www.yelp.com/biz/au-pied-de-cochon-mon...,1288,['newcanadian'],4.0,45.522169,-73.574450,$$$,"['536 Avenue Duluth E', 'Montreal, QC H2L 1A9'...",1.514281e+10
8,J6qWt6XIUmIGFHX5rQJA-w,L'Avenue,https://www.yelp.com/biz/l-avenue-montr%C3%A9al,1150,['breakfast_brunch'],4.5,45.526678,-73.580239,$$,"['922 Avenue du Mont-Royal E', 'Montreal, QC H...",1.514524e+10
...,...,...,...,...,...,...,...,...,...,...,...
539,sY0wyjh9-aMXaldNhJrP4w,Impac iTaco,https://www.yelp.com/biz/impac-itaco-montr%C3%...,1,['mexican'],4.0,45.500934,-73.573768,NaN,"['1455 Rue Peel', 'Montreal, QC H3A', 'Canada']",NaN
281,lo51FO4rKbMFBPKngoUvJw,Place Tevere,https://www.yelp.com/biz/place-tevere-montr%C3...,1,['italian'],5.0,45.499937,-73.565486,$$,"['800 Rue de la Gauchetière O', 'Montreal, QC ...",1.514871e+10
543,c7B5OHwqAJSu8QZQdLQ-Mg,La Belle Tonki,https://www.yelp.com/biz/la-belle-tonki-montr%...,1,['vietnamese'],5.0,45.540660,-73.599463,NaN,"['1335 Rue Beaubien E', 'Montreal, QC H2G 1K7'...",1.514419e+10
554,HRvuIWsjuYK6ZptATC2O1A,Caffettiera,https://www.yelp.com/biz/caffettiera-montr%C3%...,1,['cafes'],5.0,45.500785,-73.576235,NaN,"['2055 Rue Stanley', 'Montreal, QC H3A 1R7', '...",1.514507e+10


In [76]:
# request reviews
id = df.id.iloc[43]
#id = '0W4lkclzZThpx3V65bVgig'
url = f'https://api.yelp.com/v3/businesses/{id}/reviews'
r2 = requests.get(url, headers=headers, params=params)
j2 = json.loads(r2.text)
print(r.status_code)
#j2['reviews']

200


In [96]:
df2 = pd.DataFrame()
cnt = 0

# unpack a list of dict
def unpack_user(e):
    try:
        reviews['user_id'] = [ _['id'] for _ in e ]
        reviews['profile_url'] = [ _['profile_url'] for _ in e ]
        reviews['name'] = [ _['name'] for _ in e ]
    except Exception as e:
        None


while cnt < len(df):
    id = df.id.iloc[cnt]
    url = f'https://api.yelp.com/v3/businesses/{id}/reviews'
    r2 = requests.get(url, headers=headers)
    j2 = json.loads(r2.text)
    try:
        c = j2['reviews']
    except Exception as e:
        None
        
    reviews = {}
    reviews['biz_id'] = id
    reviews['rev_id'] = [ catch(lambda : e['id']) for e in c ]
    reviews['time'] = [ catch(lambda : e['time_created']) for e in c ]
    #reviews['user'] = [ unpack_user(e['user']) for e in c ]
    reviews['user'] = [ catch(lambda : e['user']) for e in c ]
    reviews['rating'] = [ catch(lambda : e['rating']) for e in c ]
    reviews['text'] = [ catch(lambda : e['text']) for e in c ]

    cnt += 1
    #print(cnt)
    df2 = pd.concat([df2, pd.DataFrame.from_dict(reviews)])

In [93]:
df2

,biz_id,rev_id,time,user,rating,text
0,NSbVtd0lzORHV7NaubG9Pg,TcdfECBle3S20RhgDvd4dw,2020-08-30 11:30:46,"{'id': '50DaCXl1VqnOrXwCH9oAdA', 'profile_url'...",5.0,"Pre-COVID visit, now I'm just writing reviews ..."
1,NSbVtd0lzORHV7NaubG9Pg,gP5Hs0dmH9HEv9DxipFcIQ,2019-12-26 17:55:39,"{'id': 'TejIW9rZaH6tWmKGSXP9nw', 'profile_url'...",5.0,Came here because my bf wanted some exotic foo...
2,NSbVtd0lzORHV7NaubG9Pg,oWW-kowodWpy-wTlJ3p_2Q,2020-09-26 21:38:47,"{'id': 'JpVRcAZDdhCHogeA8bJx8A', 'profile_url'...",5.0,A must go for Quebecois food! Delicious Pate C...
3,uxrvc9CYDJhDUqiXNVWmww,EEQ3bS3BanteVdz9By9JnQ,2020-09-28 16:19:00,"{'id': 'HV6pgnVdongJWet-dUjSpQ', 'profile_url'...",5.0,Really unique and solid experience. Coming fro...
4,uxrvc9CYDJhDUqiXNVWmww,vFCcwUTX6tPhUobaMjK1LQ,2019-07-25 18:27:39,"{'id': 'HjyML4u5LCKXMrzx_FQPKw', 'profile_url'...",5.0,It's true only 2 seatings each evening. They f...
...,...,...,...,...,...,...
443,ZSDEj6mscdZdt9CwoZo7yQ,1zcMktxqGte2w7zNudXXaA,2020-08-21 16:03:29,"{'id': 'KmoSLOLefVNbJVsJQY8j-w', 'profile_url'...",5.0,This is probably the best vegan restaurant I'v...
444,ZSDEj6mscdZdt9CwoZo7yQ,UP0ioWEQWOgBpP4FcytF4g,2019-08-20 07:41:52,"{'id': 'E8YsbKVW0DtJPsrP9Wvszg', 'profile_url'...",4.0,Very good bimibap was a nice break from the ch...
445,S0aQB-OoruhnU5vj93c2MQ,YVGb9OMqN6P9_sQ3dmyIZw,2020-08-26 19:02:15,"{'id': 'fjFYpwpvlIlqoiNa3myDEA', 'profile_url'...",4.0,"Went there a couple of times, to try different..."
446,S0aQB-OoruhnU5vj93c2MQ,TdBYEX8pDAWH81UGkob-yQ,2020-09-13 14:42:44,"{'id': 'TfLxu0ByOBszcPxcLBRGpQ', 'profile_url'...",2.0,We were shocked to discover how bad this crepe...


In [94]:
df2 = df2.reset_index(drop=True)
#df2.to_csv('data/reviews_mtl.csv', index=False)
#df2 = pd.read_csv('data/reviews_mtl.csv')
#df2.sort_values(by='rating')

In [ ]:
# theses reviews are only an exerpt of the full text so I guess I have to scrape everything